In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from google.colab import files # Colab 파일 다운로드를 위해 추가

# --- 1. 기본 설정 ---
start_date = datetime.now() - timedelta(days=5 * 365) # 기간을 5년으로 설정
end_date = datetime.now()
TOTAL_TRANSACTIONS = 2500  # 생성할 평균 거래 수

# 품목 정보 및 사용 빈도 가중치 설정
item_specs = {
    '의료 소모품': [
        ('MED-SYR-001', '주사기', 100, 10), ('MED-NEE-002', '주사 바늘', 500, 10),
        ('MED-GAU-003', '멸균 거즈', 200, 8), ('MED-SWB-004', '알콜스왑', 200, 9),
        ('MED-GLV-005', '니트릴 장갑', 100, 8), ('MED-MSK-006', '마스크', 50, 5),
        ('MED-IVK-007', '수액 키트', 50, 7)
    ],
    '일반 의약품': [
        ('OTC-PAI-008', '소염 진통제', 30, 4), ('OTC-FEV-009', '해열제', 50, 4)
    ],
    '전문 의약품': [
        ('ETC-ANT-010', '항생제(주사제)', 10, 3), ('ETC-ANE-011', '마취제', 5, 2),
        ('ETC-IVF-012', '포도당 수액', 10, 6)
    ]
}

suppliers = ['A업체', 'B업체', 'C업체']

# --- 2. 품목(items) 데이터프레임 생성 ---
items_list = []
for category, details in item_specs.items():
    for item_id, item_name, ea_per_box, weight in details:
        items_list.append({
            'item_id': item_id,
            'item_name': item_name,
            'category': category,
            'ea_per_box': ea_per_box,
            'supplier_name': random.choice(suppliers),
            'current_stock_ea': 0, # 최종 단계에서 계산
            'usage_weight': weight
        })
items_df = pd.DataFrame(items_list)

# --- 3. 거래 시뮬레이션 ---
inventory_batches_list = []
transactions_list = []
batch_id_counter = 1
transaction_id_counter = 1

# 시간 흐름에 따른 거래 생성
current_date = start_date
while len(transactions_list) < TOTAL_TRANSACTIONS and current_date < end_date:
    # 평일에만 거래 발생 (90% 확률)
    if current_date.weekday() < 5 and random.random() < 0.9:

        # 하루에 1~3건의 거래를 발생시키기 위한 반복문 추가
        for _ in range(random.randint(1, 3)):
            if len(transactions_list) >= TOTAL_TRANSACTIONS:
                break

            # 입고(15%) 또는 출고(85%) 결정
            transaction_type = random.choices(['입고', '출고'], weights=[0.15, 0.85], k=1)[0]

            if transaction_type == '입고':
                item_to_stock = items_df.sample(1).iloc[0]
                in_box_qty = random.randint(10, 50)
                ea_qty = in_box_qty * item_to_stock['ea_per_box']
                expiry_date = (current_date + timedelta(days=random.randint(270, 600))).date()

                # 배치 추가
                new_batch = {
                    'batch_id': batch_id_counter,
                    'item_id': item_to_stock['item_id'],
                    'expiry_date': expiry_date,
                    'in_date': current_date,
                    'current_batch_ea': ea_qty
                }
                inventory_batches_list.append(new_batch)

                # 거래 기록 추가
                transactions_list.append({
                    'transaction_id': transaction_id_counter,
                    'transaction_date': current_date,
                    'transaction_type': '입고',
                    'item_id': item_to_stock['item_id'],
                    'batch_id': batch_id_counter,
                    'ea_qty': ea_qty,
                    'in_box_qty': in_box_qty,
                    'out_ea_qty': None
                })
                batch_id_counter += 1
                transaction_id_counter += 1

            else: # 출고
                # 현재 재고가 있는 품목 중에서만 출고 시도
                inventory_df = pd.DataFrame(inventory_batches_list)
                if not inventory_df.empty and inventory_df['current_batch_ea'].sum() > 0:
                    available_items = inventory_df[inventory_df['current_batch_ea'] > 0]['item_id'].unique()

                    # 사용 빈도에 따라 출고 품목 선택
                    available_items_weights = items_df[items_df['item_id'].isin(available_items)]
                    if not available_items_weights.empty:
                        item_id_to_ship = available_items_weights.sample(1, weights='usage_weight')['item_id'].iloc[0]

                        required_qty = random.randint(5, 200)

                        # FIFO 로직: 유통기한, 입고일 순으로 정렬된 배치에서 차감
                        batches_for_item = inventory_df[(inventory_df['item_id'] == item_id_to_ship) & (inventory_df['current_batch_ea'] > 0)].sort_values(by=['expiry_date', 'in_date'])

                        if not batches_for_item.empty:
                            for index, batch in batches_for_item.iterrows():
                                if required_qty <= 0: break

                                ship_qty = min(required_qty, batch['current_batch_ea'])

                                # inventory_batches_list에서 직접 수정
                                for b in inventory_batches_list:
                                    if b['batch_id'] == batch['batch_id']:
                                        b['current_batch_ea'] -= ship_qty
                                        break

                                # 거래 기록
                                transactions_list.append({
                                    'transaction_id': transaction_id_counter,
                                    'transaction_date': current_date,
                                    'transaction_type': '출고',
                                    'item_id': item_id_to_ship,
                                    'batch_id': batch['batch_id'],
                                    'ea_qty': -ship_qty,
                                    'in_box_qty': None,
                                    'out_ea_qty': ship_qty
                                })
                                transaction_id_counter += 1
                                required_qty -= ship_qty

    current_date += timedelta(days=1)


# --- 4. 최종 재고 상태 조정 (긴급 발주 품목 생성) ---
inventory_df = pd.DataFrame(inventory_batches_list)
if not inventory_df.empty:
    stock_summary = inventory_df.groupby('item_id')['current_batch_ea'].sum().reset_index()

    # 재고가 가장 많은 상위 2개 품목을 찾아서 재고 대량 출고
    if len(stock_summary) > 2:
        items_to_reduce = stock_summary.nlargest(2, 'current_batch_ea')

        for _, row in items_to_reduce.iterrows():
            item_id_to_reduce = row['item_id']
            current_stock = row['current_batch_ea']

            # 재고의 85% ~ 95%를 소진시켜 긴급 발주 상황 연출
            qty_to_reduce = int(current_stock * random.uniform(0.85, 0.95))
            if qty_to_reduce <= 0: continue

            # FIFO로 출고 처리
            batches_to_reduce = inventory_df[(inventory_df['item_id'] == item_id_to_reduce) & (inventory_df['current_batch_ea'] > 0)].sort_values(by=['expiry_date', 'in_date'])

            for index, batch in batches_to_reduce.iterrows():
                if qty_to_reduce <= 0: break

                ship_qty = min(qty_to_reduce, batch['current_batch_ea'])

                for b in inventory_batches_list:
                    if b['batch_id'] == batch['batch_id']:
                        b['current_batch_ea'] -= ship_qty
                        break

                transactions_list.append({
                    'transaction_id': transaction_id_counter,
                    'transaction_date': end_date,
                    'transaction_type': '출고',
                    'item_id': item_id_to_reduce,
                    'batch_id': batch['batch_id'],
                    'ea_qty': -ship_qty,
                    'in_box_qty': None,
                    'out_ea_qty': ship_qty
                })
                transaction_id_counter += 1
                qty_to_reduce -= ship_qty


# --- 5. 최종 데이터프레임 생성 및 SQL 파일 작성 ---
inventory_batches_df = pd.DataFrame(inventory_batches_list)
transactions_df = pd.DataFrame(transactions_list)

# items 테이블의 최종 재고량 업데이트
if not inventory_batches_df.empty:
    final_stock = inventory_batches_df.groupby('item_id')['current_batch_ea'].sum().reset_index()
    items_df = items_df.drop(columns=['current_stock_ea', 'usage_weight']).merge(final_stock, on='item_id', how='left').fillna(0)
    items_df = items_df.rename(columns={'current_batch_ea': 'current_stock_ea'})
else:
    items_df['current_stock_ea'] = 0
    items_df = items_df.drop(columns=['usage_weight'])


# SQL 파일 생성
sql_file_name = 'mock_inventory_data.sql'
with open(sql_file_name, 'w', encoding='utf-8') as f:
    f.write("-- 외래 키 제약 조건 검사 일시 비활성화\n")
    f.write("SET FOREIGN_KEY_CHECKS = 0;\n\n")

    f.write("-- 기존 테이블 데이터 삭제\n")
    f.write("TRUNCATE TABLE transactions;\n")
    f.write("TRUNCATE TABLE inventory_batches;\n")
    f.write("TRUNCATE TABLE items;\n\n")

    # items 데이터 삽입
    f.write("-- 1. items 데이터 삽입\n")
    for _, row in items_df.iterrows():
        f.write(f"INSERT INTO items (item_id, item_name, category, ea_per_box, supplier_name, current_stock_ea) VALUES "
                f"('{row['item_id']}', '{row['item_name']}', '{row['category']}', {row['ea_per_box']}, '{row['supplier_name']}', {int(row['current_stock_ea'])});\n")

    # inventory_batches 데이터 삽입
    f.write("\n-- 2. inventory_batches 데이터 삽입\n")
    for _, row in inventory_batches_df.iterrows():
        f.write(f"INSERT INTO inventory_batches (batch_id, item_id, expiry_date, in_date, current_batch_ea) VALUES "
                f"({row['batch_id']}, '{row['item_id']}', '{row['expiry_date']}', '{row['in_date'].strftime('%Y-%m-%d %H:%M:%S')}', {row['current_batch_ea']});\n")

    # transactions 데이터 삽입
    f.write("\n-- 3. transactions 데이터 삽입\n")
    for _, row in transactions_df.iterrows():
        in_box_qty_str = 'NULL' if pd.isna(row['in_box_qty']) else int(row['in_box_qty'])
        out_ea_qty_str = 'NULL' if pd.isna(row['out_ea_qty']) else int(row['out_ea_qty'])
        f.write(f"INSERT INTO transactions (transaction_id, transaction_date, transaction_type, item_id, batch_id, ea_qty, in_box_qty, out_ea_qty) VALUES "
                f"({row['transaction_id']}, '{row['transaction_date'].strftime('%Y-%m-%d %H:%M:%S')}', '{row['transaction_type']}', '{row['item_id']}', {row['batch_id']}, {row['ea_qty']}, {in_box_qty_str}, {out_ea_qty_str});\n")

    f.write("\n-- 외래 키 제약 조건 검사 다시 활성화\n")
    f.write("SET FOREIGN_KEY_CHECKS = 1;\n")

print(f"`{sql_file_name}` 파일이 성공적으로 생성되었습니다.")
print(f" - 총 품목 수: {len(items_df)}")
print(f" - 총 배치 수: {len(inventory_batches_df)}")
print(f" - 총 거래 기록 수: {len(transactions_df)}")

# --- ✨ Google Colab 전용: 파일 자동 다운로드 ---
files.download(sql_file_name)

`mock_inventory_data.sql` 파일이 성공적으로 생성되었습니다.
 - 총 품목 수: 12
 - 총 배치 수: 322
 - 총 거래 기록 수: 2482


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from google.colab import files

# --- 0. SQL 스키마 및 프로시저 정의 (ENUM에 '폐기' 추가) ---
SQL_SCHEMA = """
-- 외래 키 제약 조건 검사 일시 비활성화
SET FOREIGN_KEY_CHECKS = 0;

-- 기존 테이블이 존재하면 삭제
DROP TABLE IF EXISTS transactions;
DROP TABLE IF EXISTS inventory_batches;
DROP TABLE IF EXISTS items;

-- 기존 저장 프로시저가 존재하면 삭제
DROP PROCEDURE IF EXISTS perform_fifo_shipment;

-- ------------------------------------------------------------------
-- 1단계: 테이블 생성
-- ------------------------------------------------------------------

CREATE TABLE items (
    item_id VARCHAR(50) PRIMARY KEY NOT NULL COMMENT '품목 기준 코드 (ID)',
    item_name VARCHAR(100) NOT NULL COMMENT '품목명',
    category VARCHAR(50) NOT NULL COMMENT '품목 분류',
    ea_per_box INT NOT NULL COMMENT '1박스당 EA 수량',
    supplier_name VARCHAR(50) COMMENT '업체명',
    current_stock_ea INT NOT NULL DEFAULT 0 COMMENT '현재 총 재고량 (EA 단위)'
);

CREATE TABLE inventory_batches (
    batch_id INT AUTO_INCREMENT PRIMARY KEY,
    item_id VARCHAR(50) NOT NULL,
    expiry_date DATE NOT NULL COMMENT '유통기한 (FIFO의 주요 기준)',
    in_date DATETIME NOT NULL COMMENT '입고일',
    current_batch_ea INT NOT NULL DEFAULT 0 COMMENT '해당 배치의 남은 재고 수량',
    FOREIGN KEY (item_id) REFERENCES items(item_id),
    INDEX idx_item_expiry (item_id, expiry_date, in_date)
);

CREATE TABLE transactions (
    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
    transaction_date DATETIME NOT NULL,
    transaction_type ENUM('입고', '출고', '폐기') NOT NULL, -- '폐기' 유형 추가
    item_id VARCHAR(50) NOT NULL,
    batch_id INT COMMENT '관련된 재고 배치의 ID',
    ea_qty INT NOT NULL COMMENT '거래 수량 (+입고, -출고/폐기)',
    in_box_qty INT COMMENT '입고 시 박스 수량',
    out_ea_qty INT COMMENT '출고/폐기 시 EA 수량',
    FOREIGN KEY (item_id) REFERENCES items(item_id),
    FOREIGN KEY (batch_id) REFERENCES inventory_batches(batch_id)
);
"""

SQL_PROCEDURE = """
-- ------------------------------------------------------------------
-- 2단계: 저장 프로시저 생성
-- ------------------------------------------------------------------

DELIMITER //
CREATE PROCEDURE perform_fifo_shipment (
    IN p_item_id VARCHAR(50),
    IN p_out_ea_qty INT
)
BEGIN
    DECLARE v_required_qty INT DEFAULT p_out_ea_qty;
    DECLARE v_batch_ea INT;
    DECLARE v_batch_id INT;
    DECLARE v_total_stock INT;

    START TRANSACTION;

    SELECT current_stock_ea INTO v_total_stock
    FROM items
    WHERE item_id = p_item_id
    FOR UPDATE;

    IF v_total_stock < p_out_ea_qty THEN
        ROLLBACK;
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = '재고가 부족하여 출고할 수 없습니다.';
    ELSE
        batch_loop: LOOP
            SELECT batch_id, current_batch_ea
            INTO v_batch_id, v_batch_ea
            FROM inventory_batches
            WHERE item_id = p_item_id AND current_batch_ea > 0
            ORDER BY expiry_date ASC, in_date ASC
            LIMIT 1;

            IF v_batch_id IS NULL OR v_required_qty <= 0 THEN
                LEAVE batch_loop;
            END IF;

            IF v_batch_ea >= v_required_qty THEN
                UPDATE inventory_batches
                SET current_batch_ea = current_batch_ea - v_required_qty
                WHERE batch_id = v_batch_id;

                INSERT INTO transactions (transaction_date, transaction_type, item_id, batch_id, ea_qty, out_ea_qty)
                VALUES (NOW(), '출고', p_item_id, v_batch_id, -v_required_qty, v_required_qty);

                SET v_required_qty = 0;
            ELSE
                UPDATE inventory_batches
                SET current_batch_ea = 0
                WHERE batch_id = v_batch_id;

                INSERT INTO transactions (transaction_date, transaction_type, item_id, batch_id, ea_qty, out_ea_qty)
                VALUES (NOW(), '출고', p_item_id, v_batch_id, -v_batch_ea, v_batch_ea);

                SET v_required_qty = v_required_qty - v_batch_ea;
            END IF;

            SET v_batch_id = NULL;
        END LOOP;

        UPDATE items
        SET current_stock_ea = current_stock_ea - p_out_ea_qty
        WHERE item_id = p_item_id;

        COMMIT;
    END IF;
END //
DELIMITER ;
"""


# --- 1. 기본 설정 ---
start_date = datetime.now() - timedelta(days=5 * 365)
end_date = datetime.now()
TOTAL_TRANSACTIONS = 2800  # 폐기 기록을 고려하여 목표치 상향

# 품목 정보 (이전과 동일)
item_specs = {
    '의료 소모품': [
        ('MED-SYR-001', '주사기', 100, 10), ('MED-NEE-002', '주사 바늘', 500, 10),
        ('MED-GAU-003', '멸균 거즈', 200, 8), ('MED-SWB-004', '알콜스왑', 200, 9),
        ('MED-GLV-005', '니트릴 장갑', 100, 8), ('MED-MSK-006', '마스크', 50, 5),
        ('MED-IVK-007', '수액 키트', 50, 7)
    ],
    '일반 의약품': [
        ('OTC-PAI-008', '소염 진통제', 30, 4), ('OTC-FEV-009', '해열제', 50, 4)
    ],
    '전문 의약품': [
        ('ETC-ANT-010', '항생제(주사제)', 10, 3), ('ETC-ANE-011', '마취제', 5, 2),
        ('ETC-IVF-012', '포도당 수액', 10, 6)
    ]
}
suppliers = ['A업체', 'B업체', 'C업체']

# --- 2. 품목(items) 데이터프레임 생성 ---
items_list = []
for category, details in item_specs.items():
    for item_id, item_name, ea_per_box, weight in details:
        items_list.append({
            'item_id': item_id,
            'item_name': item_name,
            'category': category,
            'ea_per_box': ea_per_box,
            'supplier_name': random.choice(suppliers),
            'current_stock_ea': 0,
            'usage_weight': weight
        })
items_df = pd.DataFrame(items_list)

# --- 3. 거래 시뮬레이션 ---
inventory_batches_list = []
transactions_list = []
batch_id_counter = 1
transaction_id_counter = 1

current_date = start_date
while len(transactions_list) < TOTAL_TRANSACTIONS and current_date < end_date:

    # --- ✨ 폐기 로직 추가 ---
    # 날짜가 지남에 따라 유통기한이 만료된 배치를 확인하고 폐기 처리
    # list 복사본으로 반복해야 안전하게 원본 리스트 수정 가능
    for batch in inventory_batches_list[:]:
        if batch['expiry_date'] < current_date.date() and batch['current_batch_ea'] > 0:
            disposed_qty = batch['current_batch_ea']

            # 폐기 거래 기록 추가
            transactions_list.append({
                'transaction_id': transaction_id_counter,
                'transaction_date': current_date,
                'transaction_type': '폐기',
                'item_id': batch['item_id'],
                'batch_id': batch['batch_id'],
                'ea_qty': -disposed_qty,
                'in_box_qty': None,
                'out_ea_qty': disposed_qty
            })
            transaction_id_counter += 1

            # 배치 재고를 0으로 변경
            batch['current_batch_ea'] = 0

    # 평일에만 입고/출고 발생 (90% 확률)
    if current_date.weekday() < 5 and random.random() < 0.9:

        for _ in range(random.randint(1, 3)):
            if len(transactions_list) >= TOTAL_TRANSACTIONS:
                break

            transaction_type = random.choices(['입고', '출고'], weights=[0.15, 0.85], k=1)[0]

            if transaction_type == '입고':
                # ... (이전과 동일한 입고 로직) ...
                item_to_stock = items_df.sample(1).iloc[0]
                in_box_qty = random.randint(10, 50)
                ea_qty = in_box_qty * item_to_stock['ea_per_box']
                expiry_date = (current_date + timedelta(days=random.randint(270, 600))).date()
                new_batch = {
                    'batch_id': batch_id_counter, 'item_id': item_to_stock['item_id'],
                    'expiry_date': expiry_date, 'in_date': current_date,
                    'current_batch_ea': ea_qty
                }
                inventory_batches_list.append(new_batch)
                transactions_list.append({
                    'transaction_id': transaction_id_counter, 'transaction_date': current_date,
                    'transaction_type': '입고', 'item_id': item_to_stock['item_id'],
                    'batch_id': batch_id_counter, 'ea_qty': ea_qty,
                    'in_box_qty': in_box_qty, 'out_ea_qty': None
                })
                batch_id_counter += 1
                transaction_id_counter += 1

            else: # 출고
                # ... (이전과 동일한 출고 로직) ...
                inventory_df = pd.DataFrame(inventory_batches_list)
                if not inventory_df.empty and inventory_df['current_batch_ea'].sum() > 0:
                    available_items = inventory_df[inventory_df['current_batch_ea'] > 0]['item_id'].unique()
                    available_items_weights = items_df[items_df['item_id'].isin(available_items)]
                    if not available_items_weights.empty:
                        item_id_to_ship = available_items_weights.sample(1, weights='usage_weight')['item_id'].iloc[0]
                        required_qty = random.randint(5, 200)
                        batches_for_item = inventory_df[(inventory_df['item_id'] == item_id_to_ship) & (inventory_df['current_batch_ea'] > 0)].sort_values(by=['expiry_date', 'in_date'])
                        if not batches_for_item.empty:
                            for index, batch in batches_for_item.iterrows():
                                if required_qty <= 0: break
                                ship_qty = min(required_qty, batch['current_batch_ea'])
                                for b in inventory_batches_list:
                                    if b['batch_id'] == batch['batch_id']:
                                        b['current_batch_ea'] -= ship_qty
                                        break
                                transactions_list.append({
                                    'transaction_id': transaction_id_counter, 'transaction_date': current_date,
                                    'transaction_type': '출고', 'item_id': item_id_to_ship,
                                    'batch_id': batch['batch_id'], 'ea_qty': -ship_qty,
                                    'in_box_qty': None, 'out_ea_qty': ship_qty
                                })
                                transaction_id_counter += 1
                                required_qty -= ship_qty

    current_date += timedelta(days=1)

# --- 4. 최종 재고 상태 조정 (이전과 동일) ---
inventory_df = pd.DataFrame(inventory_batches_list)
if not inventory_df.empty:
    stock_summary = inventory_df.groupby('item_id')['current_batch_ea'].sum().reset_index()
    if len(stock_summary) > 2:
        items_to_reduce = stock_summary.nlargest(2, 'current_batch_ea')
        for _, row in items_to_reduce.iterrows():
            item_id_to_reduce = row['item_id']
            current_stock = row['current_batch_ea']
            qty_to_reduce = int(current_stock * random.uniform(0.85, 0.95))
            if qty_to_reduce <= 0: continue
            batches_to_reduce = inventory_df[(inventory_df['item_id'] == item_id_to_reduce) & (inventory_df['current_batch_ea'] > 0)].sort_values(by=['expiry_date', 'in_date'])
            for index, batch in batches_to_reduce.iterrows():
                if qty_to_reduce <= 0: break
                ship_qty = min(qty_to_reduce, batch['current_batch_ea'])
                for b in inventory_batches_list:
                    if b['batch_id'] == batch['batch_id']:
                        b['current_batch_ea'] -= ship_qty
                        break
                transactions_list.append({
                    'transaction_id': transaction_id_counter, 'transaction_date': end_date,
                    'transaction_type': '출고', 'item_id': item_id_to_reduce,
                    'batch_id': batch['batch_id'], 'ea_qty': -ship_qty,
                    'in_box_qty': None, 'out_ea_qty': ship_qty
                })
                transaction_id_counter += 1
                qty_to_reduce -= ship_qty

# --- 5. 최종 데이터프레임 생성 및 SQL 파일 작성 ---
inventory_batches_df = pd.DataFrame(inventory_batches_list)
transactions_df = pd.DataFrame(transactions_list)

if not inventory_batches_df.empty:
    final_stock = inventory_batches_df.groupby('item_id')['current_batch_ea'].sum().reset_index()
    items_df = items_df.drop(columns=['current_stock_ea', 'usage_weight']).merge(final_stock, on='item_id', how='left').fillna(0)
    items_df = items_df.rename(columns={'current_batch_ea': 'current_stock_ea'})
else:
    items_df['current_stock_ea'] = 0
    items_df = items_df.drop(columns=['usage_weight'])

sql_file_name = 'complete_setup.sql'
with open(sql_file_name, 'w', encoding='utf-8') as f:
    f.write(SQL_SCHEMA)
    f.write(SQL_PROCEDURE)

    f.write("\n-- ------------------------------------------------------------------\n")
    f.write("-- 3단계: 초기 데이터 삽입\n")
    f.write("-- ------------------------------------------------------------------\n\n")

    f.write("-- items 데이터 삽입\n")
    for _, row in items_df.iterrows():
        f.write(f"INSERT INTO items (item_id, item_name, category, ea_per_box, supplier_name, current_stock_ea) VALUES "
                f"('{row['item_id']}', '{row['item_name']}', '{row['category']}', {row['ea_per_box']}, '{row['supplier_name']}', {int(row['current_stock_ea'])});\n")

    f.write("\n-- inventory_batches 데이터 삽입\n")
    for _, row in inventory_batches_df.iterrows():
        f.write(f"INSERT INTO inventory_batches (batch_id, item_id, expiry_date, in_date, current_batch_ea) VALUES "
                f"({row['batch_id']}, '{row['item_id']}', '{row['expiry_date']}', '{row['in_date'].strftime('%Y-%m-%d %H:%M:%S')}', {row['current_batch_ea']});\n")

    f.write("\n-- transactions 데이터 삽입\n")
    for _, row in transactions_df.iterrows():
        in_box_qty_str = 'NULL' if pd.isna(row['in_box_qty']) else int(row['in_box_qty'])
        out_ea_qty_str = 'NULL' if pd.isna(row['out_ea_qty']) else int(row['out_ea_qty'])
        f.write(f"INSERT INTO transactions (transaction_id, transaction_date, transaction_type, item_id, batch_id, ea_qty, in_box_qty, out_ea_qty) VALUES "
                f"({row['transaction_id']}, '{row['transaction_date'].strftime('%Y-%m-%d %H:%M:%S')}', '{row['transaction_type']}', '{row['item_id']}', {row['batch_id']}, {row['ea_qty']}, {in_box_qty_str}, {out_ea_qty_str});\n")

    f.write("\n-- 외래 키 제약 조건 검사 다시 활성화\n")
    f.write("SET FOREIGN_KEY_CHECKS = 1;\n")

print(f"`{sql_file_name}` 파일이 성공적으로 생성되었습니다.")
print(f" - 총 품목 수: {len(items_df)}")
print(f" - 총 배치 수: {len(inventory_batches_df)}")
print(f" - 총 거래 기록 수: {len(transactions_df)}")

# --- ✨ Google Colab 전용: 파일 자동 다운로드 ---
files.download(sql_file_name)

`complete_setup.sql` 파일이 성공적으로 생성되었습니다.
 - 총 품목 수: 12
 - 총 배치 수: 350
 - 총 거래 기록 수: 2617


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from google.colab import files

# --- 0. SQL 스키마 및 프로시저 정의 (ENUM에 '폐기' 추가) ---
SQL_SCHEMA = """
-- 외래 키 제약 조건 검사 일시 비활성화
SET FOREIGN_KEY_CHECKS = 0;

-- 기존 테이블이 존재하면 삭제
DROP TABLE IF EXISTS transactions;
DROP TABLE IF EXISTS inventory_batches;
DROP TABLE IF EXISTS items;

-- 기존 저장 프로시저가 존재하면 삭제
DROP PROCEDURE IF EXISTS perform_fifo_shipment;

-- ------------------------------------------------------------------
-- 1단계: 테이블 생성
-- ------------------------------------------------------------------

CREATE TABLE items (
    item_id VARCHAR(50) PRIMARY KEY NOT NULL COMMENT '품목 기준 코드 (ID)',
    item_name VARCHAR(100) NOT NULL COMMENT '품목명',
    category VARCHAR(50) NOT NULL COMMENT '품목 분류',
    ea_per_box INT NOT NULL COMMENT '1박스당 EA 수량',
    supplier_name VARCHAR(50) COMMENT '업체명',
    current_stock_ea INT NOT NULL DEFAULT 0 COMMENT '현재 총 재고량 (EA 단위)'
);

CREATE TABLE inventory_batches (
    batch_id INT AUTO_INCREMENT PRIMARY KEY,
    item_id VARCHAR(50) NOT NULL,
    expiry_date DATE NOT NULL COMMENT '유통기한 (FIFO의 주요 기준)',
    in_date DATETIME NOT NULL COMMENT '입고일',
    current_batch_ea INT NOT NULL DEFAULT 0 COMMENT '해당 배치의 남은 재고 수량',
    FOREIGN KEY (item_id) REFERENCES items(item_id),
    INDEX idx_item_expiry (item_id, expiry_date, in_date)
);

CREATE TABLE transactions (
    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
    transaction_date DATETIME NOT NULL,
    transaction_type ENUM('입고', '출고', '폐기') NOT NULL, -- '폐기' 유형 추가
    item_id VARCHAR(50) NOT NULL,
    batch_id INT COMMENT '관련된 재고 배치의 ID',
    ea_qty INT NOT NULL COMMENT '거래 수량 (+입고, -출고/폐기)',
    in_box_qty INT COMMENT '입고 시 박스 수량',
    out_ea_qty INT COMMENT '출고/폐기 시 EA 수량',
    FOREIGN KEY (item_id) REFERENCES items(item_id),
    FOREIGN KEY (batch_id) REFERENCES inventory_batches(batch_id)
);
"""

SQL_PROCEDURE = """
-- ------------------------------------------------------------------
-- 2단계: 저장 프로시저 생성
-- ------------------------------------------------------------------

DELIMITER //
CREATE PROCEDURE perform_fifo_shipment (
    IN p_item_id VARCHAR(50),
    IN p_out_ea_qty INT
)
BEGIN
    DECLARE v_required_qty INT DEFAULT p_out_ea_qty;
    DECLARE v_batch_ea INT;
    DECLARE v_batch_id INT;
    DECLARE v_total_stock INT;

    START TRANSACTION;

    SELECT current_stock_ea INTO v_total_stock
    FROM items
    WHERE item_id = p_item_id
    FOR UPDATE;

    IF v_total_stock < p_out_ea_qty THEN
        ROLLBACK;
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = '재고가 부족하여 출고할 수 없습니다.';
    ELSE
        batch_loop: LOOP
            SELECT batch_id, current_batch_ea
            INTO v_batch_id, v_batch_ea
            FROM inventory_batches
            WHERE item_id = p_item_id AND current_batch_ea > 0
            ORDER BY expiry_date ASC, in_date ASC
            LIMIT 1;

            IF v_batch_id IS NULL OR v_required_qty <= 0 THEN
                LEAVE batch_loop;
            END IF;

            IF v_batch_ea >= v_required_qty THEN
                UPDATE inventory_batches
                SET current_batch_ea = current_batch_ea - v_required_qty
                WHERE batch_id = v_batch_id;

                INSERT INTO transactions (transaction_date, transaction_type, item_id, batch_id, ea_qty, out_ea_qty)
                VALUES (NOW(), '출고', p_item_id, v_batch_id, -v_required_qty, v_required_qty);

                SET v_required_qty = 0;
            ELSE
                UPDATE inventory_batches
                SET current_batch_ea = 0
                WHERE batch_id = v_batch_id;

                INSERT INTO transactions (transaction_date, transaction_type, item_id, batch_id, ea_qty, out_ea_qty)
                VALUES (NOW(), '출고', p_item_id, v_batch_id, -v_batch_ea, v_batch_ea);

                SET v_required_qty = v_required_qty - v_batch_ea;
            END IF;

            SET v_batch_id = NULL;
        END LOOP;

        UPDATE items
        SET current_stock_ea = current_stock_ea - p_out_ea_qty
        WHERE item_id = p_item_id;

        COMMIT;
    END IF;
END //
DELIMITER ;
"""


# --- 1. 기본 설정 ---
start_date = datetime.now() - timedelta(days=5 * 365)
end_date = datetime.now()
TOTAL_TRANSACTIONS = 2800

# --- ✨ 현실적인 데이터로 수정 ---
item_specs = {
    '의료 소모품': [
        # (item_id, item_name, ea_per_box, usage_weight)
        ('MED-SYR-001', '주사기 (10ml)', 100, 12),
        ('MED-NEE-002', '주사 바늘 (25G)', 100, 12),
        ('MED-GAU-003', '멸균 거즈 (10x10cm)', 200, 9),
        ('MED-SWB-004', '알콜스왑', 200, 10),
        ('MED-GLV-005', '니트릴 장갑 (M)', 100, 8),
        ('MED-MSK-006', '마스크 (KF94)', 50, 5),
        ('MED-IVK-007', '수액 키트', 50, 7)
    ],
    '일반 의약품': [
        ('OTC-PAI-008', '소염 진통제 (타이레놀)', 30, 4),
        ('OTC-FEV-009', '해열제 (부루펜)', 50, 4)
    ],
    '전문 의약품': [
        ('ETC-ANT-010', '항생제(주사제)', 10, 3),
        ('ETC-ANE-011', '마취제 (리도카인)', 5, 2),
        ('ETC-IVF-012', '포도당 수액 (5%)', 10, 6)
    ]
}
suppliers = ['A제약', 'B메디칼', 'C헬스케어']

# --- 2. 품목(items) 데이터프레임 생성 ---
items_list = []
for category, details in item_specs.items():
    for item_id, item_name, ea_per_box, weight in details:
        items_list.append({
            'item_id': item_id,
            'item_name': item_name,
            'category': category,
            'ea_per_box': ea_per_box,
            'supplier_name': random.choice(suppliers),
            'current_stock_ea': 0,
            'usage_weight': weight
        })
items_df = pd.DataFrame(items_list)

# --- 3. 거래 시뮬레이션 ---
inventory_batches_list = []
transactions_list = []
batch_id_counter = 1
transaction_id_counter = 1

current_date = start_date
while len(transactions_list) < TOTAL_TRANSACTIONS and current_date < end_date:

    # --- 폐기 로직 ---
    for batch in inventory_batches_list[:]:
        if batch['expiry_date'] < current_date.date() and batch['current_batch_ea'] > 0:
            disposed_qty = batch['current_batch_ea']
            transactions_list.append({
                'transaction_id': transaction_id_counter,
                'transaction_date': current_date, 'transaction_type': '폐기',
                'item_id': batch['item_id'], 'batch_id': batch['batch_id'],
                'ea_qty': -disposed_qty, 'in_box_qty': None, 'out_ea_qty': disposed_qty
            })
            transaction_id_counter += 1
            batch['current_batch_ea'] = 0

    # 평일에만 입고/출고 발생
    if current_date.weekday() < 5 and random.random() < 0.95:

        for _ in range(random.randint(1, 3)):
            if len(transactions_list) >= TOTAL_TRANSACTIONS: break

            transaction_type = random.choices(['입고', '출고'], weights=[0.2, 0.8], k=1)[0]

            if transaction_type == '입고':
                item_to_stock = items_df.sample(1).iloc[0]
                in_box_qty = random.randint(5, 20) # 입고 박스량 조절
                ea_qty = in_box_qty * item_to_stock['ea_per_box']
                # 유통기한 1개월 ~ 1년 사이로 설정
                expiry_date = (current_date + timedelta(days=random.randint(30, 365))).date()
                new_batch = {
                    'batch_id': batch_id_counter, 'item_id': item_to_stock['item_id'],
                    'expiry_date': expiry_date, 'in_date': current_date,
                    'current_batch_ea': ea_qty
                }
                inventory_batches_list.append(new_batch)
                transactions_list.append({
                    'transaction_id': transaction_id_counter, 'transaction_date': current_date,
                    'transaction_type': '입고', 'item_id': item_to_stock['item_id'],
                    'batch_id': batch_id_counter, 'ea_qty': ea_qty,
                    'in_box_qty': in_box_qty, 'out_ea_qty': None
                })
                batch_id_counter += 1
                transaction_id_counter += 1

            else: # 출고
                inventory_df = pd.DataFrame(inventory_batches_list)
                if not inventory_df.empty and inventory_df['current_batch_ea'].sum() > 0:
                    available_items = inventory_df[inventory_df['current_batch_ea'] > 0]['item_id'].unique()

                    # 소모품 사용량 가중치 높여서 출고
                    weights_df = items_df[items_df['item_id'].isin(available_items)]
                    if not weights_df.empty:
                        item_id_to_ship = weights_df.sample(1, weights='usage_weight')['item_id'].iloc[0]
                        required_qty = random.randint(10, 300)

                        batches_for_item = inventory_df[(inventory_df['item_id'] == item_id_to_ship) & (inventory_df['current_batch_ea'] > 0)].sort_values(by=['expiry_date', 'in_date'])
                        if not batches_for_item.empty:
                            for index, batch in batches_for_item.iterrows():
                                if required_qty <= 0: break
                                ship_qty = min(required_qty, batch['current_batch_ea'])
                                for b in inventory_batches_list:
                                    if b['batch_id'] == batch['batch_id']:
                                        b['current_batch_ea'] -= ship_qty
                                        break
                                transactions_list.append({
                                    'transaction_id': transaction_id_counter, 'transaction_date': current_date,
                                    'transaction_type': '출고', 'item_id': item_id_to_ship,
                                    'batch_id': batch['batch_id'], 'ea_qty': -ship_qty,
                                    'in_box_qty': None, 'out_ea_qty': ship_qty
                                })
                                transaction_id_counter += 1
                                required_qty -= ship_qty

    current_date += timedelta(days=1)

# --- 4. 최종 재고 상태 조정 (긴급 발주 품목 생성) ---
inventory_df = pd.DataFrame(inventory_batches_list)
if not inventory_df.empty:
    stock_summary = inventory_df.groupby('item_id')['current_batch_ea'].sum().reset_index()
    if len(stock_summary) > 2:
        items_to_reduce = stock_summary.nlargest(2, 'current_batch_ea')
        for _, row in items_to_reduce.iterrows():
            item_id_to_reduce = row['item_id']
            current_stock = row['current_batch_ea']
            # 재고의 90% ~ 98%를 소진시켜 더 타이트한 상황 연출
            qty_to_reduce = int(current_stock * random.uniform(0.90, 0.98))
            if qty_to_reduce <= 0: continue

            batches_to_reduce = inventory_df[(inventory_df['item_id'] == item_id_to_reduce) & (inventory_df['current_batch_ea'] > 0)].sort_values(by=['expiry_date', 'in_date'])
            for index, batch in batches_to_reduce.iterrows():
                if qty_to_reduce <= 0: break
                ship_qty = min(qty_to_reduce, batch['current_batch_ea'])
                for b in inventory_batches_list:
                    if b['batch_id'] == batch['batch_id']:
                        b['current_batch_ea'] -= ship_qty
                        break
                transactions_list.append({
                    'transaction_id': transaction_id_counter, 'transaction_date': end_date,
                    'transaction_type': '출고', 'item_id': item_id_to_reduce,
                    'batch_id': batch['batch_id'], 'ea_qty': -ship_qty,
                    'in_box_qty': None, 'out_ea_qty': ship_qty
                })
                transaction_id_counter += 1
                qty_to_reduce -= ship_qty

# --- 5. 최종 데이터프레임 생성 및 SQL 파일 작성 ---
inventory_batches_df = pd.DataFrame(inventory_batches_list)
transactions_df = pd.DataFrame(transactions_list)

if not inventory_batches_df.empty:
    final_stock = inventory_batches_df.groupby('item_id')['current_batch_ea'].sum().reset_index()
    items_df = items_df.drop(columns=['current_stock_ea', 'usage_weight']).merge(final_stock, on='item_id', how='left').fillna(0)
    items_df = items_df.rename(columns={'current_batch_ea': 'current_stock_ea'})
else:
    items_df['current_stock_ea'] = 0
    items_df = items_df.drop(columns=['usage_weight'])

sql_file_name = 'complete_setup.sql'
with open(sql_file_name, 'w', encoding='utf-8') as f:
    f.write(SQL_SCHEMA)
    f.write(SQL_PROCEDURE)

    f.write("\n-- ------------------------------------------------------------------\n")
    f.write("-- 3단계: 초기 데이터 삽입\n")
    f.write("-- ------------------------------------------------------------------\n\n")

    f.write("-- items 데이터 삽입\n")
    for _, row in items_df.iterrows():
        f.write(f"INSERT INTO items (item_id, item_name, category, ea_per_box, supplier_name, current_stock_ea) VALUES "
                f"('{row['item_id']}', '{row['item_name']}', '{row['category']}', {row['ea_per_box']}, '{row['supplier_name']}', {int(row['current_stock_ea'])});\n")

    f.write("\n-- inventory_batches 데이터 삽입\n")
    for _, row in inventory_batches_df.iterrows():
        f.write(f"INSERT INTO inventory_batches (batch_id, item_id, expiry_date, in_date, current_batch_ea) VALUES "
                f"({row['batch_id']}, '{row['item_id']}', '{row['expiry_date']}', '{row['in_date'].strftime('%Y-%m-%d %H:%M:%S')}', {row['current_batch_ea']});\n")

    f.write("\n-- transactions 데이터 삽입\n")
    for _, row in transactions_df.iterrows():
        in_box_qty_str = 'NULL' if pd.isna(row['in_box_qty']) else int(row['in_box_qty'])
        out_ea_qty_str = 'NULL' if pd.isna(row['out_ea_qty']) else int(row['out_ea_qty'])
        f.write(f"INSERT INTO transactions (transaction_id, transaction_date, transaction_type, item_id, batch_id, ea_qty, in_box_qty, out_ea_qty) VALUES "
                f"({row['transaction_id']}, '{row['transaction_date'].strftime('%Y-%m-%d %H:%M:%S')}', '{row['transaction_type']}', '{row['item_id']}', {row['batch_id']}, {row['ea_qty']}, {in_box_qty_str}, {out_ea_qty_str});\n")

    f.write("\n-- 외래 키 제약 조건 검사 다시 활성화\n")
    f.write("SET FOREIGN_KEY_CHECKS = 1;\n")

print(f"`{sql_file_name}` 파일이 성공적으로 생성되었습니다.")
print(f" - 총 품목 수: {len(items_df)}")
print(f" - 총 배치 수: {len(inventory_batches_df)}")
print(f" - 총 거래 기록 수: {len(transactions_df)}")

# --- ✨ Google Colab 전용: 파일 자동 다운로드 ---
files.download(sql_file_name)

`complete_setup.sql` 파일이 성공적으로 생성되었습니다.
 - 총 품목 수: 12
 - 총 배치 수: 496
 - 총 거래 기록 수: 2806


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from google.colab import files

# --- 0. SQL 스키마 및 프로시저 정의 (ENUM에 '폐기' 추가) ---
SQL_SCHEMA = """
-- 외래 키 제약 조건 검사 일시 비활성화
SET FOREIGN_KEY_CHECKS = 0;

-- 기존 테이블이 존재하면 삭제
DROP TABLE IF EXISTS transactions;
DROP TABLE IF EXISTS inventory_batches;
DROP TABLE IF EXISTS items;

-- 기존 저장 프로시저가 존재하면 삭제
DROP PROCEDURE IF EXISTS perform_fifo_shipment;

-- ------------------------------------------------------------------
-- 1단계: 테이블 생성
-- ------------------------------------------------------------------

CREATE TABLE items (
    item_id VARCHAR(50) PRIMARY KEY NOT NULL COMMENT '품목 기준 코드 (ID)',
    item_name VARCHAR(100) NOT NULL COMMENT '품목명',
    category VARCHAR(50) NOT NULL COMMENT '품목 분류',
    ea_per_box INT NOT NULL COMMENT '1박스당 EA 수량',
    supplier_name VARCHAR(50) COMMENT '업체명',
    current_stock_ea INT NOT NULL DEFAULT 0 COMMENT '현재 총 재고량 (EA 단위)'
);

CREATE TABLE inventory_batches (
    batch_id INT AUTO_INCREMENT PRIMARY KEY,
    item_id VARCHAR(50) NOT NULL,
    expiry_date DATE NOT NULL COMMENT '유통기한 (FIFO의 주요 기준)',
    in_date DATETIME NOT NULL COMMENT '입고일',
    current_batch_ea INT NOT NULL DEFAULT 0 COMMENT '해당 배치의 남은 재고 수량',
    FOREIGN KEY (item_id) REFERENCES items(item_id),
    INDEX idx_item_expiry (item_id, expiry_date, in_date)
);

CREATE TABLE transactions (
    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
    transaction_date DATETIME NOT NULL,
    transaction_type ENUM('입고', '출고', '폐기') NOT NULL, -- '폐기' 유형 추가
    item_id VARCHAR(50) NOT NULL,
    batch_id INT COMMENT '관련된 재고 배치의 ID',
    ea_qty INT NOT NULL COMMENT '거래 수량 (+입고, -출고/폐기)',
    in_box_qty INT COMMENT '입고 시 박스 수량',
    out_ea_qty INT COMMENT '출고/폐기 시 EA 수량',
    FOREIGN KEY (item_id) REFERENCES items(item_id),
    FOREIGN KEY (batch_id) REFERENCES inventory_batches(batch_id)
);
"""

SQL_PROCEDURE = """
-- ------------------------------------------------------------------
-- 2단계: 저장 프로시저 생성
-- ------------------------------------------------------------------

DELIMITER //
CREATE PROCEDURE perform_fifo_shipment (
    IN p_item_id VARCHAR(50),
    IN p_out_ea_qty INT
)
BEGIN
    DECLARE v_required_qty INT DEFAULT p_out_ea_qty;
    DECLARE v_batch_ea INT;
    DECLARE v_batch_id INT;
    DECLARE v_total_stock INT;

    START TRANSACTION;

    SELECT current_stock_ea INTO v_total_stock
    FROM items
    WHERE item_id = p_item_id
    FOR UPDATE;

    IF v_total_stock < p_out_ea_qty THEN
        ROLLBACK;
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = '재고가 부족하여 출고할 수 없습니다.';
    ELSE
        batch_loop: LOOP
            SELECT batch_id, current_batch_ea
            INTO v_batch_id, v_batch_ea
            FROM inventory_batches
            WHERE item_id = p_item_id AND current_batch_ea > 0
            ORDER BY expiry_date ASC, in_date ASC
            LIMIT 1;

            IF v_batch_id IS NULL OR v_required_qty <= 0 THEN
                LEAVE batch_loop;
            END IF;

            IF v_batch_ea >= v_required_qty THEN
                UPDATE inventory_batches
                SET current_batch_ea = current_batch_ea - v_required_qty
                WHERE batch_id = v_batch_id;

                INSERT INTO transactions (transaction_date, transaction_type, item_id, batch_id, ea_qty, out_ea_qty)
                VALUES (NOW(), '출고', p_item_id, v_batch_id, -v_required_qty, v_required_qty);

                SET v_required_qty = 0;
            ELSE
                UPDATE inventory_batches
                SET current_batch_ea = 0
                WHERE batch_id = v_batch_id;

                INSERT INTO transactions (transaction_date, transaction_type, item_id, batch_id, ea_qty, out_ea_qty)
                VALUES (NOW(), '출고', p_item_id, v_batch_id, -v_batch_ea, v_batch_ea);

                SET v_required_qty = v_required_qty - v_batch_ea;
            END IF;

            SET v_batch_id = NULL;
        END LOOP;

        UPDATE items
        SET current_stock_ea = current_stock_ea - p_out_ea_qty
        WHERE item_id = p_item_id;

        COMMIT;
    END IF;
END //
DELIMITER ;
"""


# --- 1. 기본 설정 ---
start_date = datetime.now() - timedelta(days=5 * 365)
end_date = datetime.now()
TOTAL_TRANSACTIONS = 2800

# --- ✨ 현실적인 데이터로 수정 ---
item_specs = {
    '의료 소모품': [
        # (item_id, item_name, ea_per_box, usage_weight)
        ('MED-SYR-001', '주사기 (10ml)', 100, 12),
        ('MED-NEE-002', '주사 바늘 (25G)', 100, 12),
        ('MED-GAU-003', '멸균 거즈 (10x10cm)', 200, 9),
        ('MED-SWB-004', '알콜스왑', 200, 10),
        ('MED-GLV-005', '니트릴 장갑 (M)', 100, 8),
        ('MED-MSK-006', '마스크 (KF94)', 50, 5),
        ('MED-IVK-007', '수액 키트', 50, 7)
    ],
    '일반 의약품': [
        ('OTC-PAI-008', '소염 진통제 (타이레놀)', 30, 4),
        ('OTC-FEV-009', '해열제 (부루펜)', 50, 4)
    ],
    '전문 의약품': [
        ('ETC-ANT-010', '항생제(주사제)', 10, 3),
        ('ETC-ANE-011', '마취제 (리도카인)', 5, 2),
        ('ETC-IVF-012', '포도당 수액 (5%)', 10, 6)
    ]
}
suppliers = ['A제약', 'B메디칼', 'C헬스케어']

# --- 2. 품목(items) 데이터프레임 생성 ---
items_list = []
for category, details in item_specs.items():
    for item_id, item_name, ea_per_box, weight in details:
        items_list.append({
            'item_id': item_id,
            'item_name': item_name,
            'category': category,
            'ea_per_box': ea_per_box,
            'supplier_name': random.choice(suppliers),
            'current_stock_ea': 0,
            'usage_weight': weight
        })
items_df = pd.DataFrame(items_list)

# --- 3. 거래 시뮬레이션 ---
inventory_batches_list = []
transactions_list = []
batch_id_counter = 1
transaction_id_counter = 1

current_date = start_date
while len(transactions_list) < TOTAL_TRANSACTIONS and current_date < end_date:

    # --- 폐기 로직 (수정된 부분) ---
    batches_to_dispose = [
        batch for batch in inventory_batches_list
        if batch['expiry_date'] < current_date.date() and batch['current_batch_ea'] > 0
    ]

    for batch in batches_to_dispose:
        disposed_qty = batch['current_batch_ea']
        transactions_list.append({
            'transaction_id': transaction_id_counter,
            'transaction_date': current_date, 'transaction_type': '폐기',
            'item_id': batch['item_id'], 'batch_id': batch['batch_id'],
            'ea_qty': -disposed_qty, 'in_box_qty': None, 'out_ea_qty': disposed_qty
        })
        transaction_id_counter += 1
        batch['current_batch_ea'] = 0 # 👈 원본 리스트의 딕셔너리 재고를 0으로 설정

    # 평일에만 입고/출고 발생
    if current_date.weekday() < 5 and random.random() < 0.95:

        for _ in range(random.randint(1, 3)):
            if len(transactions_list) >= TOTAL_TRANSACTIONS: break

            transaction_type = random.choices(['입고', '출고'], weights=[0.2, 0.8], k=1)[0]

            if transaction_type == '입고':
                item_to_stock = items_df.sample(1).iloc[0]
                in_box_qty = random.randint(5, 20) # 입고 박스량 조절
                ea_qty = in_box_qty * item_to_stock['ea_per_box']
                # 유통기한 1개월 ~ 1년 사이로 설정
                expiry_date = (current_date + timedelta(days=random.randint(30, 365))).date()
                new_batch = {
                    'batch_id': batch_id_counter, 'item_id': item_to_stock['item_id'],
                    'expiry_date': expiry_date, 'in_date': current_date,
                    'current_batch_ea': ea_qty
                }
                inventory_batches_list.append(new_batch)
                transactions_list.append({
                    'transaction_id': transaction_id_counter, 'transaction_date': current_date,
                    'transaction_type': '입고', 'item_id': item_to_stock['item_id'],
                    'batch_id': batch_id_counter, 'ea_qty': ea_qty,
                    'in_box_qty': in_box_qty, 'out_ea_qty': None
                })
                batch_id_counter += 1
                transaction_id_counter += 1

            else: # 출고
                # 출고 로직은 inventory_batches_list를 직접 업데이트하므로, 변경 없음
                inventory_df = pd.DataFrame(inventory_batches_list)
                if not inventory_df.empty and inventory_df['current_batch_ea'].sum() > 0:
                    available_items = inventory_df[inventory_df['current_batch_ea'] > 0]['item_id'].unique()

                    # 소모품 사용량 가중치 높여서 출고
                    weights_df = items_df[items_df['item_id'].isin(available_items)]
                    if not weights_df.empty:
                        item_id_to_ship = weights_df.sample(1, weights='usage_weight')['item_id'].iloc[0]
                        required_qty = random.randint(10, 300)

                        batches_for_item = inventory_df[(inventory_df['item_id'] == item_id_to_ship) & (inventory_df['current_batch_ea'] > 0)].sort_values(by=['expiry_date', 'in_date'])
                        if not batches_for_item.empty:
                            for index, batch in batches_for_item.iterrows():
                                if required_qty <= 0: break
                                ship_qty = min(required_qty, batch['current_batch_ea'])
                                for b in inventory_batches_list:
                                    if b['batch_id'] == batch['batch_id']:
                                        b['current_batch_ea'] -= ship_qty
                                        break
                                transactions_list.append({
                                    'transaction_id': transaction_id_counter, 'transaction_date': current_date,
                                    'transaction_type': '출고', 'item_id': item_id_to_ship,
                                    'batch_id': batch['batch_id'], 'ea_qty': -ship_qty,
                                    'in_box_qty': None, 'out_ea_qty': ship_qty
                                })
                                transaction_id_counter += 1
                                required_qty -= ship_qty

    current_date += timedelta(days=1)

# --- 4. 최종 재고 상태 조정 (긴급 발주 품목 생성) ---
# 이 부분도 inventory_batches_list를 직접 업데이트합니다.
inventory_df = pd.DataFrame(inventory_batches_list)
if not inventory_df.empty:
    stock_summary = inventory_df.groupby('item_id')['current_batch_ea'].sum().reset_index()
    if len(stock_summary) > 2:
        items_to_reduce = stock_summary.nlargest(2, 'current_batch_ea')
        for _, row in items_to_reduce.iterrows():
            item_id_to_reduce = row['item_id']
            current_stock = row['current_batch_ea']
            # 재고의 90% ~ 98%를 소진시켜 더 타이트한 상황 연출
            qty_to_reduce = int(current_stock * random.uniform(0.90, 0.98))
            if qty_to_reduce <= 0: continue

            batches_to_reduce = inventory_df[(inventory_df['item_id'] == item_id_to_reduce) & (inventory_df['current_batch_ea'] > 0)].sort_values(by=['expiry_date', 'in_date'])
            for index, batch in batches_to_reduce.iterrows():
                if qty_to_reduce <= 0: break
                ship_qty = min(qty_to_reduce, batch['current_batch_ea'])
                for b in inventory_batches_list:
                    if b['batch_id'] == batch['batch_id']:
                        b['current_batch_ea'] -= ship_qty
                        break
                transactions_list.append({
                    'transaction_id': transaction_id_counter, 'transaction_date': end_date,
                    'transaction_type': '출고', 'item_id': item_id_to_reduce,
                    'batch_id': batch['batch_id'], 'ea_qty': -ship_qty,
                    'in_box_qty': None, 'out_ea_qty': ship_qty
                })
                transaction_id_counter += 1
                qty_to_reduce -= ship_qty

# --- 4. 최종 재고 상태 조정 (긴급 발주 품목 생성) ---
# ... (기존 4단계 로직 전체) ...
# ----------------------------------------------------------------------

# --- 4.5. 최종 폐기 정리 (추가된 부분) ---
# 시뮬레이션 종료 시점(end_date) 기준으로 유통기한 지난 재고를 최종 정리
final_dispose_date = end_date
batches_to_dispose_final = [
    batch for batch in inventory_batches_list
    if batch['expiry_date'] < final_dispose_date.date() and batch['current_batch_ea'] > 0
]

for batch in batches_to_dispose_final:
    disposed_qty = batch['current_batch_ea']
    transactions_list.append({
        'transaction_id': transaction_id_counter,
        'transaction_date': final_dispose_date, 'transaction_type': '폐기',
        'item_id': batch['item_id'], 'batch_id': batch['batch_id'],
        'ea_qty': -disposed_qty, 'in_box_qty': None, 'out_ea_qty': disposed_qty
    })
    transaction_id_counter += 1
    batch['current_batch_ea'] = 0 # 재고를 0으로 설정
# ----------------------------------------------------------------------

# --- 5. 최종 데이터프레임 생성 및 SQL 파일 작성 ---
# 최종 inventory_batches_df는 항상 최신 inventory_batches_list를 반영
inventory_batches_df = pd.DataFrame(inventory_batches_list)
transactions_df = pd.DataFrame(transactions_list)

if not inventory_batches_df.empty:
    final_stock = inventory_batches_df.groupby('item_id')['current_batch_ea'].sum().reset_index()
    items_df = items_df.drop(columns=['current_stock_ea', 'usage_weight']).merge(final_stock, on='item_id', how='left').fillna(0)
    items_df = items_df.rename(columns={'current_batch_ea': 'current_stock_ea'})
else:
    items_df['current_stock_ea'] = 0
    items_df = items_df.drop(columns=['usage_weight'])

sql_file_name = 'complete_setup.sql'
with open(sql_file_name, 'w', encoding='utf-8') as f:
    f.write(SQL_SCHEMA)
    f.write(SQL_PROCEDURE)

    f.write("\n-- ------------------------------------------------------------------\n")
    f.write("-- 3단계: 초기 데이터 삽입\n")
    f.write("-- ------------------------------------------------------------------\n\n")

    f.write("-- items 데이터 삽입\n")
    for _, row in items_df.iterrows():
        f.write(f"INSERT INTO items (item_id, item_name, category, ea_per_box, supplier_name, current_stock_ea) VALUES "
                f"('{row['item_id']}', '{row['item_name']}', '{row['category']}', {row['ea_per_box']}, '{row['supplier_name']}', {int(row['current_stock_ea'])});\n")

    f.write("\n-- inventory_batches 데이터 삽입\n")
    for _, row in inventory_batches_df.iterrows():
        f.write(f"INSERT INTO inventory_batches (batch_id, item_id, expiry_date, in_date, current_batch_ea) VALUES "
                f"({row['batch_id']}, '{row['item_id']}', '{row['expiry_date']}', '{row['in_date'].strftime('%Y-%m-%d %H:%M:%S')}', {row['current_batch_ea']});\n")

    f.write("\n-- transactions 데이터 삽입\n")
    for _, row in transactions_df.iterrows():
        in_box_qty_str = 'NULL' if pd.isna(row['in_box_qty']) else int(row['in_box_qty'])
        out_ea_qty_str = 'NULL' if pd.isna(row['out_ea_qty']) else int(row['out_ea_qty'])
        f.write(f"INSERT INTO transactions (transaction_id, transaction_date, transaction_type, item_id, batch_id, ea_qty, in_box_qty, out_ea_qty) VALUES "
                f"({row['transaction_id']}, '{row['transaction_date'].strftime('%Y-%m-%d %H:%M:%S')}', '{row['transaction_type']}', '{row['item_id']}', {row['batch_id']}, {row['ea_qty']}, {in_box_qty_str}, {out_ea_qty_str});\n")

    f.write("\n-- 외래 키 제약 조건 검사 다시 활성화\n")
    f.write("SET FOREIGN_KEY_CHECKS = 1;\n")

print(f"`{sql_file_name}` 파일이 성공적으로 생성되었습니다.")
print(f" - 총 품목 수: {len(items_df)}")
print(f" - 총 배치 수: {len(inventory_batches_df)}")
print(f" - 총 거래 기록 수: {len(transactions_df)}")

# --- ✨ Google Colab 전용: 파일 자동 다운로드 ---
files.download(sql_file_name)

`complete_setup.sql` 파일이 성공적으로 생성되었습니다.
 - 총 품목 수: 12
 - 총 배치 수: 495
 - 총 거래 기록 수: 2815


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>